# IMPORT

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import math

# METHOD

### Euclidean Distance(For Numpy Array)

In [2]:
def np_euclidean(e1,e2):
    return math.sqrt(math.fsum(list(map(lambda x: x**2 ,e1-e2))))

### Generate Data and Results for each File

In [3]:
def read_all_lines(file) :
    with open(file, 'rt') as fd:
        convert = [e.split() for e in fd.readlines()]
        all_data = []
        all_results = []
        for line in convert:
            data = line
            result = data.pop(-1)
            all_data.append(data)
            all_results.append(result)
            
        float_data = []
        for element in all_data:
            float_data.append([float(e) for e in element])
        float_data = np.array(float_data)
        all_results = np.array(all_results)
        
        normalized_data = []
        for element in float_data:
            mean = math.fsum(element)/len(element)
            std = math.sqrt(sum((element-mean)**2)/(len(element)-1))
            normalized_data.append((element - mean)/std)
        normalized_data = np.array(normalized_data)
        return normalized_data, all_results

### Dynamic Time Warping

In [4]:
def dtw_distance(s1, s2):
    DTW={}

    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

### DBA

In [5]:
from __future__ import division
from functools import reduce


__author__ ="Francois Petitjean"

def performDBA(series, n_iterations=10):
    n_series = len(series)
    max_length = reduce(max, map(len, series))

    cost_mat = np.zeros((max_length, max_length))
    delta_mat = np.zeros((max_length, max_length))
    path_mat = np.zeros((max_length, max_length), dtype=np.int8)

    medoid_ind = approximate_medoid_index(series,cost_mat,delta_mat)
    center = series[medoid_ind]

    for i in range(0,n_iterations):
        center = DBA_update(center, series, cost_mat, path_mat, delta_mat)

    return center

def approximate_medoid_index(series,cost_mat,delta_mat):
    if len(series)<=50:
        indices = range(0,len(series))
    else:
        indices = np.random.choice(range(0,len(series)),50,replace=False)

    medoid_ind = -1
    best_ss = 1e20
    for index_candidate in indices:
        candidate = series[index_candidate]
        ss = sum_of_squares(candidate,series,cost_mat,delta_mat)
        if(medoid_ind==-1 or ss<best_ss):
            best_ss = ss
            medoid_ind = index_candidate
    return medoid_ind

def sum_of_squares(s,series,cost_mat,delta_mat):
    return sum(map(lambda t:squared_DTW(s,t,cost_mat,delta_mat),series))

def DTW(s,t,cost_mat,delta_mat):
    return np.sqrt(squared_DTW(s,t,cost_mat,delta_mat))

def squared_DTW(s,t,cost_mat,delta_mat):
    s_len = len(s)
    t_len = len(t)
    length = len(s)
    fill_delta_mat_dtw(s, t, delta_mat)
    cost_mat[0, 0] = delta_mat[0, 0]
    for i in range(1, s_len):
        cost_mat[i, 0] = cost_mat[i-1, 0]+delta_mat[i, 0]

    for j in range(1, t_len):
        cost_mat[0, j] = cost_mat[0, j-1]+delta_mat[0, j]

    for i in range(1, s_len):
        for j in range(1, t_len):
            diag,left,top =cost_mat[i-1, j-1], cost_mat[i, j-1], cost_mat[i-1, j]
            if(diag <=left):
                if(diag<=top):
                    res = diag
                else:
                    res = top
            else:
                if(left<=top):
                    res = left
                else:
                    res = top
            cost_mat[i, j] = res+delta_mat[i, j]
    return cost_mat[s_len-1,t_len-1]

def fill_delta_mat_dtw(center, s, delta_mat):
    slim = delta_mat[:len(center),:len(s)]
    np.subtract.outer(center, s,out=slim)
    np.square(slim, out=slim)

def DBA_update(center, series, cost_mat, path_mat, delta_mat):
    options_argmin = [(-1, -1), (0, -1), (-1, 0)]
    updated_center = np.zeros(center.shape)
    n_elements = np.array(np.zeros(center.shape), dtype=int)
    center_length = len(center)
    for s in series:
        s_len = len(s)
        fill_delta_mat_dtw(center, s, delta_mat)
        cost_mat[0, 0] = delta_mat[0, 0]
        path_mat[0, 0] = -1

        for i in range(1, center_length):
            cost_mat[i, 0] = cost_mat[i-1, 0]+delta_mat[i, 0]
            path_mat[i, 0] = 2

        for j in range(1, s_len):
            cost_mat[0, j] = cost_mat[0, j-1]+delta_mat[0, j]
            path_mat[0, j] = 1

        for i in range(1, center_length):
            for j in range(1, s_len):
                diag,left,top =cost_mat[i-1, j-1], cost_mat[i, j-1], cost_mat[i-1, j]
                if(diag <=left):
                    if(diag<=top):
                        res = diag
                        path_mat[i,j] = 0
                    else:
                        res = top
                        path_mat[i,j] = 2
                else:
                    if(left<=top):
                        res = left
                        path_mat[i,j] = 1
                    else:
                        res = top
                        path_mat[i,j] = 2

                cost_mat[i, j] = res+delta_mat[i, j]

        i = center_length-1
        j = s_len-1

        while(path_mat[i, j] != -1):
            updated_center[i] += s[j]
            n_elements[i] += 1
            move = options_argmin[path_mat[i, j]]
            i += move[0]
            j += move[1]
        assert(i == 0 and j == 0)
        updated_center[i] += s[j]
        n_elements[i] += 1

    return np.divide(updated_center, n_elements)

### Calculate F1 Score

In [6]:
def score(test_data,test_results,mrcp_avg,noise_avg):
    e_results = list()
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for each_test_data in test_data:
        e_class = None
        min_dist = float('inf')
        for avg in mrcp_avg:
            dist = np_euclidean(each_test_data, avg) 
            if dist < min_dist:
                min_dist = dist
                e_class = 'MRCP'
        for avg in noise_avg:
            dist = np_euclidean(each_test_data, avg) 
            if dist < min_dist:
                min_dist = dist
                e_class = 'Noise'
        e_results.append(e_class)
    e_results = np.array(e_results)
    for i in range(len(test_data)):
        if e_results[i] == 'MRCP' and test_results[i] == 'MRCP' :
            TP += 1
        elif e_results[i] == 'MRCP' and test_results[i] == 'Noise' :
            FP += 1
        elif e_results[i] == 'Noise' and test_results[i] == 'MRCP' :
            FN += 1
        elif e_results[i] == 'Noise' and test_results[i] == 'Noise' :
            TN += 1
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    accuracy = (TP+TN)/(TP+TN+FP+FN)*100
    f1_score = (2*precision*recall)/(precision+recall)*100
    return accuracy,f1_score

# IMPORT DATA

### Generate Data and Results for each File

In [9]:
loop = True
threshold = 0
while(loop):
    print('---------------------------------------Threshold {}---------------------------------------'.format(threshold))
    for i in range(1,10):
        data , results = read_all_lines('training data/participant_'+str(i)+'.txt')
        test_data , test_results = read_all_lines('test data/participant_'+str(i)+'.txt')
        print('Data Participant:',i,data[:0])

        mrcp_data = list()
        noise_data = list()
        mrcp_result = list()
        noise_result = list()

        for each_data, each_result in zip(data,results):
            if each_result == 'MRCP':
                mrcp_data.append(each_data)
                mrcp_result.append(each_result)
            else:
                noise_data.append(each_data)
                noise_result.append(each_result)

        mrcp_data = np.array(mrcp_data)
        noise_data = np.array(noise_data)
        mrcp_results = np.array(mrcp_result)
        noise_results = np.array(noise_result)

        sum_mrcp_list = list()
        for each_mrcp_data in mrcp_data:
            sum_mrcp_list.append(math.fsum(each_mrcp_data))
        avg_mrcp = float(math.fsum(sum_mrcp_list)/len(mrcp_data))

        sum_noise_list = list()
        for each_noise_data in noise_data:
            sum_noise_list.append(math.fsum(each_noise_data))
        avg_noise = float(math.fsum(sum_noise_list)/len(noise_data))

#         print('Avg_MRCP: ', avg_mrcp, '\nAvg_Noise: ', avg_noise)

        mrcp_pivot = -1
        mrcp_min = float('inf')
        for i in range(len(sum_mrcp_list)):
            abs_subs = abs(sum_mrcp_list[i] - avg_mrcp)
            if  abs_subs < mrcp_min :
                mrcp_pivot = i
                mrcp_min = abs_subs

        noise_pivot = -1
        noise_min = float('inf')
        for i in range(len(sum_noise_list)):
            abs_subs = abs(sum_noise_list[i] - avg_noise)
            if  abs_subs < noise_min :
                noise_pivot = i
                noise_min = abs_subs

#         print('MRCP_pivot: ', mrcp_pivot, '\nNoise_pivot: ', noise_pivot)

        #MRCP

        dist = []
        for each_mrcp_data,idx in zip(mrcp_data,range(len(mrcp_data))):
            dist.append([np_euclidean(each_mrcp_data,mrcp_data[mrcp_pivot]),idx])

        sorted_dist = sorted(dist,key=lambda x:x[0])
#         print('Sorted_Dist_MRCP: ',sorted_dist[:3])

        diff = []
        for i in range(1,len(sorted_dist)):
            diff.append(sorted_dist[i][0]-sorted_dist[i-1][0]);
        diff = np.array(diff)
#         print('Diff_MRCP: ',diff[:3])

        T_mean = math.fsum(diff)/len(diff)
        T_std = math.sqrt(math.fsum((diff-T_mean)**2)/(len(diff)-1)) 
        T = T_std/2
#         print('T_MRCP: ',T)

        Class = []
        temp_c = []
        temp_c.append(sorted_dist[0][1])
        for i in range(len(diff)):
            if(diff[i] > T):
                Class.append(temp_c)
                temp_c = []
                temp_c.append(sorted_dist[i+1][1])
            else:
                temp_c.append(sorted_dist[i+1][1])

    #     plt.figure(figsize=(30,len(Class)))
    #     count = 1
    #     for each_class in Class:
    #         index = []
    #         plt.subplot(len(Class)/4+1,4,count)
    #         for idx in each_class:
    #             x = range(len(mrcp_data[idx]))
    #             plt.plot(x,mrcp_data[idx])
    #             index.append(idx)
    #         #plt.xlabel('MRCP : '+str(index))
    #         count += 1
    #     plt.show()

        filter_class = list(filter(lambda x:len(x)>threshold,Class))
        mrcp_avg = []
        for i in range(len(filter_class)):
            l = []
            for e in filter_class[i]:
                l.append(mrcp_data[e])
            mrcp_avg.append(performDBA(l))

    #     plt.figure(figsize=(30,len(mrcp_avg)))
    #     count = 1
    #     for each_class in mrcp_avg:
    #         plt.subplot(len(mrcp_avg)/4+1,4,count)
    #         x = range(len(each_class))
    #         plt.plot(x,each_class)
    #         count += 1
    #     plt.show()

        #Noise

        dist = []
        for each_noise_data,idx in zip(noise_data,range(len(noise_data))):
            dist.append([np_euclidean(each_noise_data,noise_data[noise_pivot]),idx])

        sorted_dist = sorted(dist,key=lambda x:x[0])
#         print('Sorted_Dist_Noise: ',sorted_dist[:3])

        diff = []
        for i in range(1,len(sorted_dist)):
            diff.append(sorted_dist[i][0]-sorted_dist[i-1][0]);
        diff = np.array(diff)
#         print('Diff_Noise: ',diff[:3])

        T_mean = math.fsum(diff)/len(diff)
        T_std = math.sqrt(math.fsum((diff-T_mean)**2)/(len(diff)-1)) 
        T = T_std/2
#         print('T_Noise: ',T)

        Class = []
        temp_c = []
        temp_c.append(sorted_dist[0][1])
        for i in range(len(diff)):
            if(diff[i] > T):
                Class.append(temp_c)
                temp_c = []
                temp_c.append(sorted_dist[i+1][1])
            else:
                temp_c.append(sorted_dist[i+1][1])

    #     plt.figure(figsize=(30,len(Class)))
    #     count = 1
    #     for each_class in Class:
    #         index = []
    #         plt.subplot(len(Class)/4+1,4,count)
    #         for idx in each_class:
    #             x = range(len(noise_data[idx]))
    #             plt.plot(x,noise_data[idx])
    #             index.append(idx)
    #         #plt.xlabel('Noise : '+str(index))
    #         count += 1
    #     plt.show()

        filter_class = list(filter(lambda x:len(x)>threshold,Class))
        noise_avg = []
        for i in range(len(filter_class)):
            l = []
            for e in filter_class[i]:
                l.append(noise_data[e])
            noise_avg.append(performDBA(l))

    #     plt.figure(figsize=(30,len(noise_avg)))
    #     count = 1
    #     for each_class in noise_avg:
    #         plt.subplot(len(noise_avg)/4+1,4,count)
    #         x = range(len(each_class))
    #         plt.plot(x,each_class)
    #         count += 1
    #     plt.show()

        print('Number of MRCP Class: ',len(mrcp_avg))
        print('Number of Noise Class: ',len(noise_avg))
        if(len(mrcp_avg)==0 or len(noise_avg)==0):
            print('Zero SubClass')
            print('------------------------')
            loop = False
            break;
        accuracy,f1_score = score(test_data,test_results,mrcp_avg,noise_avg);
        print('F1_score: ', f1_score)
        print('------------------------')
    threshold +=1

---------------------------------------Threshold 0---------------------------------------
Data Participant: 1 []
Number of MRCP Class:  9
Number of Noise Class:  46
F1_score:  38.095238095238095
------------------------
Data Participant: 2 []
Number of MRCP Class:  57
Number of Noise Class:  44
F1_score:  94.01709401709401
------------------------
Data Participant: 3 []
Number of MRCP Class:  43
Number of Noise Class:  20
F1_score:  88.13559322033898
------------------------
Data Participant: 4 []
Number of MRCP Class:  33
Number of Noise Class:  32
F1_score:  58.74125874125874
------------------------
Data Participant: 5 []
Number of MRCP Class:  34
Number of Noise Class:  22
F1_score:  78.04878048780488
------------------------
Data Participant: 6 []
Number of MRCP Class:  23
Number of Noise Class:  16
F1_score:  48.0
------------------------
Data Participant: 7 []
Number of MRCP Class:  36
Number of Noise Class:  9
F1_score:  66.66666666666667
------------------------
Data Participa